In [ ]:
!pip install huggingface_hub transformers bitsandbytes datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -U transformers accelerate bitsandbytes torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 31.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.2
    Uninstalling transformers-4.50.2:
      Successfully uninstalled transformers-4.50.2
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.5.2
    Uninstalling accelerate-1.5.2:
      Successfully uninstalled accelerate-1.5.2


## 모델 불러오기

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import IA3Config, get_peft_model, prepare_model_for_kbit_training

# 4비트 양자화
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# IA3 설정
ia3_config = IA3Config(
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    feedforward_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    task_type="CAUSAL_LM"
)

# 모델명
model_name = 'google/gemma-3-4b-it'

# 모델 불러오기
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)

# IA3를 모델에 적용
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, ia3_config)

# 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(model_name, token=True)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/modeling_auto.py:1682: FutureWarning: Loading a multimodal model with `AutoModelForCausalLM` is deprecated and will be removed in v5. `AutoModelForCausalLM` will be used to load only the text-to-text generation module.
  warnings.warn(
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

### SFT 데이터

In [ ]:
# google drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/LLM/planty/data

/content/drive/MyDrive/LLM/planty/data


### 데이터 불러오기

In [ ]:
import json
from datasets import Dataset

#json 데이터 로드
with open('sft_dataset_cleaned.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

dataset = Dataset.from_dict({"question": [item["question"] for item in data], "answer": [item["answer"] for item in data]})

In [ ]:
dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 1242
})

In [ ]:
dataset['question'][:10]

['분갈이는 얼마에 한 번씩 하나요?',
 '분갈이에 가장 좋은 시기는?',
 '분갈이가 필요한 때를 어떻게 알 수 있나요?',
 '분갈이에 필요한 도구와 용기의 선택: 도구',
 '분갈이에 필요한 도구와 용기의 선택: 용기',
 '분갈이에 필요한 도구와 용기의 선택: 용기와 식물의 비율',
 '분갈이에 필요한 도구와 용기의 선택: 용기(화분)의 선택',
 '분갈이에 어떤 흙을 사용해야 하는가?',
 '분갈이할 때 산이나 주변에서 볼 수 있는 흙도 가능한가? 아니면, 꼭 구입한 토양만 써야 하는가?',
 '분갈이 시 흙의 양은?']

In [ ]:
dataset['answer'][:5]

['성장이 왕성하여 1년마다 사는 집이 비좁아지는 식물은 매년 봄에, 2년 정도 되어야새집이 필요한 식물은 2년마다, 성장이 아주 느린 식물은 필요한 시기에 해 준다.',
 '식물도 동물처럼 휴식이 필요하며 대부분의 식물은 한겨울을 휴식기로 선택한다. 그러나 봄이 오면 다시 새로운 생명 활동을 시작하므로 아직 꽃망울이 맺히기 전에 분갈이가 필요한 식물들은 분갈이를 한다. 그러나 너무 이른 봄보다는 좀 더 따뜻해졌을때(4~5월경) 새 집으로 이사하고 싶어 하는 식물들도 많다.',
 '분갈이가 필요할 때 식물들은 열심히 신호를 보낸다. 뿌리가 배수공을 빠져나와 소리를 지르는 광경은 흔히 볼 수 있는 풍경이다. 또한 물을 줄 때마다 이미 너무 커 버려 작은 화분이 자꾸 넘어지는 모습에서 우리는 식물의신호를 읽을 수 있다.',
 '새롭게 옮겨 심을 화분, 모종삽, 막대기, 물뿌리개, 배양토, 화분망 등',
 '식물에게 용기의 선택은 사람에게 살 집을 선택하는 것과 같다. 우리는 미관상 보기에만 좋은 집이 살기 편하다고 이야기하지는 않는다. 용기는 화분 안에 들어갈 흙의 온도와 수분의 유지, 조절에도 관여하며 식물을 더욱 아름답게, 편안하게 만드는 역할도 한다.']

### 토크나이징

In [ ]:
def tokenize_function(examples):
    # 프롬프트 생성
    prompt = [f"<|system|>\n식물에 관한 질문에 정확하게 답변해주세요.</|system|>\n<|user|>\n{q}</|user|>\n<|assistant|>\n{a}</|assistant|>"
             for q, a in zip(examples["question"], examples["answer"])]

    # 토크나이징 (truncation과 padding 적용)
    tokenized = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)

    # labels 생성 (padding 부분은 -100 처리)
    tokenized["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in ids]
        for ids in tokenized["input_ids"]
    ]

    return tokenized

# 데이터셋을 토크나이징 적용하여 변환
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1242 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1242
})

## 파인튜닝

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir = "./gemma-3-4b-it-planty-ia3",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    eval_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    push_to_hub=False,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    max_grad_norm=1.0,
)

# 데이터셋 분할
train_val_split = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_data = train_val_split["train"]
val_data = train_val_split["test"]

# Trainer 정의 (data_collator 추가)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator
)

# 모델 학습 시작
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mailharu365 (mailharu365-dankook-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


It is strongly recommended to train Gemma3 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,No log,3.166465
2,No log,3.088589
3,No log,3.074288


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=105, training_loss=12.563888113839285, metrics={'train_runtime': 2422.5824, 'train_samples_per_second': 1.383, 'train_steps_per_second': 0.043, 'total_flos': 3.731192305724621e+16, 'train_loss': 12.563888113839285, 'epoch': 3.0})

## 모델 저장

In [ ]:
model.save_pretrained('./gemma-3-4b-it-planty-ia3')
tokenizer.save_pretrained("./gemma-3-4b-it-planty-ia3")

('./gemma-3-4b-it-planty-ia3/tokenizer_config.json',
 './gemma-3-4b-it-planty-ia3/special_tokens_map.json',
 './gemma-3-4b-it-planty-ia3/tokenizer.model',
 './gemma-3-4b-it-planty-ia3/added_tokens.json',
 './gemma-3-4b-it-planty-ia3/tokenizer.json')

In [ ]:
from huggingface_hub import login, create_repo
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. Hugging Face 로그인
login(token='your_api')

# 2. 저장소 생성 (저장소가 없다면 생성)
repo_id = "yerim00/gemma-3-4b-it-planty-ia3"  # 원하는 repo_id 입력
create_repo(repo_id, private=False)  # 공개 저장소 생성 (비공개는 private=True)

# 3. 모델과 토크나이저 로드 (로컬 경로)
model_path = './gemma-3-4b-it-planty-ia3'  # 로컬에 저장된 모델 경로
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 4. 모델과 토크나이저를 Hugging Face Hub에 업로드
model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/modeling_auto.py:1682: FutureWarning: Loading a multimodal model with `AutoModelForCausalLM` is deprecated and will be removed in v5. `AutoModelForCausalLM` will be used to load only the text-to-text generation module.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/1.73M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/yerim00/gemma-3-4b-it-planty-ia3/commit/37657f4c429ce5b4295468c64fa3038471ea8f16', commit_message='Upload tokenizer', commit_description='', oid='37657f4c429ce5b4295468c64fa3038471ea8f16', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yerim00/gemma-3-4b-it-planty-ia3', endpoint='https://huggingface.co', repo_type='model', repo_id='yerim00/gemma-3-4b-it-planty-ia3'), pr_revision=None, pr_num=None)

### 저장한 모델 불러오기

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = 'yerim00/gemma-3-4b-it-planty-ia3'

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

adapter_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/modeling_auto.py:1682: FutureWarning: Loading a multimodal model with `AutoModelForCausalLM` is deprecated and will be removed in v5. `AutoModelForCausalLM` will be used to load only the text-to-text generation module.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/1.73M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [ ]:
# 입력 프롬프트 설정
prompt = "다육식물은 겨울철에 어떻게 관리해야 하나요?"

# 입력을 토큰화
inputs = tokenizer(prompt, return_tensors="pt")

# 모델을 사용하여 텍스트 생성
output = model.generate(**inputs, max_length=200, do_sample=True, top_k=50, top_p=0.95)

# 생성된 텍스트 디코딩
plant = tokenizer.decode(output[0], skip_special_tokens=True)

print("번역:")
print(plant)

번역:
다육식물은 겨울철에 어떻게 관리해야 하나요?

다육식물 겨울철 관리법에 대해 자세히 알려드리겠습니다. 다육식물은 건조한 환경에 적응했기 때문에 겨울철에는 특히 주의를 기울여야 합니다.

**1. 온도 관리**

*   **최적 온도:** 다육식물 종류에 따라 다르지만, 일반적으로 5°C ~ 15°C 정도가 적당합니다.
*   **낮반:** 밤에는 0°C까지 내려가도 괜찮지만, 최저 온도가 너무 낮으면 뿌리가 손상될 수 있습니다.
*   **햇빛:** 겨울철에는 햇빛이 부족하므로, 햇빛이 잘 드는 곳으로 옮겨주거나 식물 생장 LED 등을 활용하여 보조 조명을 제공합니다.
*   **실내 관리:** 실내에서 관리할 경우, 온도 변화가 심하지 않은


In [ ]:
import torch
import time

device = torch.device("cpu")

# 추론 최적화
def generate_answer(prompt, max_length=50):
    # 시간 측정 시작
    start_time = time.time()

    # 토큰화
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # 추론 시 그라디언트 계산 비활성화 (추론 성능 최적화)
    with torch.no_grad():
        # 텍스트 생성
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.7,  # 생성의 다양성 제어
            top_p=0.9,        # nucleus sampling
            top_k=50,         # 상위 k개 토큰 샘플링
            do_sample=True    # 샘플링을 통해 더 자연스러운 답변 생성
        )

    # 출력 디코딩
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 시간 측정 종료
    end_time = time.time()

    # 소요된 시간 출력
    print(f"답변 생성 시간: {end_time - start_time:.4f} 초")

    return generated_text

# 예시 사용
prompt = "다육식물은 겨울철에 어떻게 관리해야 하나요?"
answer = generate_answer(prompt)
print("생성된 답변:", answer)


답변 생성 시간: 13.6587 초
생성된 답변: 다육식물은 겨울철에 어떻게 관리해야 하나요?

다육식물 겨울철 관리법은 다음과 같습니다.

**1. 온도 관리:**

*   **최적 온도:** 다육식물은 추위에


In [ ]:
import torch
import time
from transformers import AutoTokenizer, AutoModelForCausalLM

# 입력 프롬프트 설정
prompt = "다육식물은 겨울철에 어떻게 관리해야 하나요?"

# 시간 측정 시작
start_time = time.time()

# 입력을 토큰화
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# 모델을 사용하여 텍스트 생성
output = model.generate(**inputs, max_length=200, do_sample=True, top_k=50, top_p=0.95)

# 생성된 텍스트 디코딩
plant = tokenizer.decode(output[0], skip_special_tokens=True)

# 시간 측정 종료
end_time = time.time()

# 소요된 시간 출력
print(f"답변 생성 시간: {end_time - start_time:.4f} 초")

# 생성된 답변 출력
print("번역:")
print(plant)

답변 생성 시간: 69.6366 초
번역:
다육식물은 겨울철에 어떻게 관리해야 하나요?

다육식물은 건조한 환경을 좋아하기 때문에 겨울철에도 특별한 관리가 필요합니다. 다음은 다육식물을 겨울철에 관리하는 방법입니다.

**1. 온도 관리:**

*   **적정 온도:** 대부분의 다육식물은 5°C ~ 15°C 정도의 서늘한 환경에서 잘 자랍니다.
*   **보온:** 0°C 이하의 낮은 온도에 노출되면 뿌리가 손상될 수 있으므로, 냉해를 입지 않도록 주의해야 합니다.
*   **햇빛:** 겨울철 햇빛은 봄보다 약하므로, 다육식물이 햇빛을 충분히 받을 수 있도록 창가에 배치하는 것이 좋습니다. 직사광선은 피하고, 간접광이 잘 드는 곳에 두는 것이 좋습니다.
*   **온
